In [1]:
import pandas as pd
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [2]:
#Create database values
snr=['+6db', '0db', '-6db']
machine=['slider']
id=['id_00', 'id_02', 'id_04', 'id_06']
condition=['abnormal', 'normal']

In [3]:
rows=[[a, b, c,d, e ]  for a in snr for b in machine for c in id for d in condition for e in os.listdir('E:\\'+a+'/'+b+'/'+c+'/'+d)]


In [4]:
#Create df general information
df = pd.DataFrame(rows, columns=["snr", "machine", "id","condition", "file"])

In [5]:
#Create path for each audio file, path will we use by Librosa to retrieve features. 
df['path']='E:\\'+df.snr+'/'+df.machine+'/'+df.id+'/'+df.condition+'/'+df.file


In [6]:
#get y values, sr is always 16000 (accordign to paper)
def load_sound(p):
    y, sr = librosa.load(p, sr=None)
    return y

In [7]:
df['waves_array']=df.path.apply(load_sound)

In [8]:
df.to_csv('slider_general.csv', index=False)

In [84]:
# get fast Fourier transform (FFT) 
#not added to the features_df
def ftt(y):
    ft = np.abs(librosa.stft(y[:2048], hop_length = 2048+1))
    return ft

In [85]:
#get spectral centroid
#not added to the features_df
def centroid(y, sr):
    cent=librosa.feature.spectral_centroid(y=y, sr=sr)
    return cent

In [91]:
#features tested
def chromagram(y, sr):
    spec=np.abs(librosa.stft(y))
    chroma=np.mean(librosa.feature.chroma_stft(S=spec, sr=sr).T,axis=0)
    return chroma

def melspec(y, sr):
    mel=np.mean(librosa.feature.melspectrogram(y=y, sr=sr).T,axis=0)
    return mel
 
def mfcc(y, sr):
    mfc=np.mean(librosa.feature.mfcc(y=y, sr=sr).T, axis=0)
    return mfc

In [88]:
def get_features(y):
    sr=160000
    chroma=chromagram(y, sr=sr)
    mel=melspec(y, sr=sr)
    mfc_coef=mfcc(y, sr=sr)
    feature_matrix=np.array([])
    feature_matrix=np.hstack((chroma, mel, mfc_coef))
    return feature_matrix

In [89]:
def load_features():
    X, y = [], []
    for i in df.path.values:
        wave=load_sound(i)
        features=get_features(wave)
        X.append(features)
        if i.split('/')[3] == 'normal':
            target= int(1)
        else:
            target=int(0)
        y.append(target)
    return np.array(X), np.array(y)

In [92]:
features, condition = load_features()


In [99]:
df_features=pd.DataFrame(features)
df_target=pd.DataFrame(condition)

0:11 -> chromagram ( 12 columns)  <br />
12:number?-> melspec ( 128 columns)<br />
number? : number? -> mfcc ( 20 columns)

In [100]:
features_df.to_csv('features_df.csv', index=False)
target_df.to_csv('target_df.csv', index=False)


# ML start

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_features, 
    df_target, 
    test_size=0.2, 
    random_state=69
)

In [ ]:
classification_models = [
    KNeighborsClassifier(),
    SVC(kernel='linear'),
    SVC(kernel='rbf'),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [ ]:
scores = []
for model in classification_models:
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    model_name = type(model).__name__
    if model_name=='SVC' and model.kernel=='rbf': model_name+=' RBF kernel'
    scores.append((model_name,(f'{100*score:.2f}%')))
# Make it pretty
scores_df = pd.DataFrame(scores,columns=['Classifier','Accuracy Score'])
scores_df.sort_values(by='Accuracy Score',axis=0,ascending=False)